In [1]:
from bs4 import BeautifulSoup
import csv
import re
import os 
import pandas as pd
import glob

In [2]:
# reading html files from folder

In [3]:
data = []

#'Riv_Shelter_HTML_Files'is the name for the shelter HTML Files
folder_path = 'Riv_Food_Pantry_HTML_Files'

# Loop through each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.html'):
        file_path = os.path.join(folder_path, filename)

        with open(file_path, 'r', encoding='utf-8') as file:
            html_content = file.read()

        soup = BeautifulSoup(html_content, 'html.parser')


        # Find the <ul> element with class "best-programs"
        ul_element = soup.find('ul', {'class': 'best-programs'})

        # Find all <li> elements within the <ul>
        li_elements = ul_element.find_all('li', {'class': 'search-result card card-v3 program-info'})

        # Loop through the found <li> elements and extract labels and values
        for li_element in li_elements:
            # Use BeautifulSoup to parse the HTML of each li_element
            li_soup = BeautifulSoup(str(li_element), 'html.parser')

            # Find the program heading 
            program_heading_element = li_soup.find('div', {'class': 'card-heading'})
            Service_name = program_heading_element.find('a', {'class': 'activity-log ph-flyout-click cwdc-flyout-click click-cookie'}).text
            print('Service name: ', Service_name)
    
            # Extract program URL
            program_url_element = program_heading_element.find('a', {'class': 'activity-log ph-flyout-click cwdc-flyout-click click-cookie'})
            program_url = program_url_element['href'] if program_url_element else None
            Service_url = "https://www.auntbertha.com/"+program_url
            print('URL: ', Service_url)
            
            is_reviewed = False

            # Reviewed on
            Reviewd_on_element = program_heading_element.find('div', {'class': 'last-reviewed'})
            if Reviewd_on_element:  # Check if the element exists
                Reviewd_on_text = Reviewd_on_element.get_text(strip=True)  # Clean up the text
                match = re.search(r'\d{2}/\d{2}/\d{4}', Reviewd_on_text)
                Reviewd_on = match.group() if match else None
                if Reviewd_on:  # If we found a review date
                    is_reviewed = True  # Set the flag to True
                print("Reviewed on:", Reviewd_on)
            else:
                Reviewd_on = None  # Set to None if not found
                print("No review date found.")
    
            # Access class="program-tags"
            program_tags = li_soup.find('div', {'class': 'program-tags'})

            # Main Services
            main_s = []
            main_service_list = program_tags.find('ul', {'class': 'list-inline'})
            main_service_items = main_service_list.find_all('li')

            for item in main_service_items:
                main_service = item.find('a', {'class': 'loading-on-click activity-log'}).text.strip()
                main_s.append(main_service)

            print("Main Services:", main_s)

            # Try to access the "Other Services" section
            other_service_list = program_tags.find('div', {'class': 'secondary-tags'})
            if other_service_list:
                other_s = []
                other_service_items = other_service_list.find('ul', {'class': 'list-inline'}).find_all('li')

                for item in other_service_items:
                    other_service = item.find('a', {'class': 'loading-on-click activity-log'}).text.strip()
                    other_s.append(other_service)
            else:
                other_s = None

            print("Other Services:", other_s)

            # Access the "Serving" section
            serving_section = program_tags.find('div', {'class': 'attribute-tags'})

            # Initialize a list to store the serving information
            serving_ = []
    
            # Find all the <li> elements within the serving section
            serving_items = serving_section.find_all('li')

            # Iterate through the serving items and extract the text from the <a> elements
            for item in serving_items:
                link = item.find('a', {'class': 'loading-on-click activity-log'})
    
                # Check if the link was found
                if link:  # Only proceed if the link exists
                    serving_text = link.text.strip()
                    serving_.append(serving_text)

            print("Serving:", serving_)

    
            # accessing next-steps-module, extract phone number, location, hours 
    
            next_steps_module = li_soup.find('div', {'class': 'next-steps-module'})

            # Extract phone number
            phone_number_elements = next_steps_module.find_all('span', {'class': 'result-next-step-item'})

            # Also find all 'a' elements with href attributes containing 'tel:'
            tel_link_elements = next_steps_module.find_all('a', href=True)

            phone_number = None  # Initialize phone number as None

            # Extract phone numbers from text-based spans
            for element in phone_number_elements:
                phone_number_text = element.text.strip() if element else None
    
                # Regex to extract digits (handling separators like spaces or hyphens)
                phone_number_matches = re.findall(r'[\d-]+', phone_number_text)
    
                # Join the digits into a single phone number string if matches are found
                if phone_number_matches:
                    phone_number = ''.join(phone_number_matches)
                    break  # Stop once we've found a phone number

            # If no phone number found from text, check the 'tel:' href links
            if not phone_number:
                for element in tel_link_elements:
                    href_value = element['href']
        
                    # Check if the href contains 'tel:' and extract digits
                    if 'tel:' in href_value:
                        phone_number_matches = re.findall(r'[\d-]+', href_value)
                        if phone_number_matches:
                            phone_number = ''.join(phone_number_matches)
                            break  # Stop once we've found a phone number

            # Print the extracted phone number
            if phone_number:
                print("Phone Number:", phone_number)
            else:
                print("No phone number found.")

            # Extract location address
            location_address_element = next_steps_module.find('a', {'class': 'activity-log ph-flyout-click cwdc-flyout-click map-link with-address'})
            location_address = location_address_element.text.strip() if location_address_element else None
            location_address = re.sub(r'\s+', ' ', location_address) if location_address_element else None
            print("Location Address:", location_address)

            # Extract URL
            location_url_map = location_address_element['href'] if location_address_element else None
            print("Location url map:", location_url_map)
            
            hours_info = {
                '24_hour': False,
                'Monday': None,
                'Tuesday': None,
                'Wednesday': None,
                'Thursday': None,
                'Friday': None,
                'Saturday': None,
                'Sunday': None
            }
            
            # Extract hours based on structure
            hours_element = next_steps_module.find('div', {'class': 'office-hours-schedule see-hours-dropdown'})
            if hours_element:
                # Find all the <span> elements within the hours_element
                day_spans = hours_element.find_all('span')

                # Ensure that we have even pairs of day and corresponding hours
                if len(day_spans) % 2 == 0:
                    # Iterate through the spans in pairs (day and its corresponding hours)
                    for i in range(0, len(day_spans), 2):
                        day_span = day_spans[i]          # This is the day (e.g., "Monday:")
                        hours_span = day_spans[i + 1]    # This is the corresponding hours (e.g., "Closed" or time)

                        # Extract the day name and remove the colon
                        day = day_span.text.strip()[:-1]  # Remove the ':' at the end
                        full_day = day.strip()            # Ensure there's no extra space

                        # Check if the day is valid and exists in hours_info
                        if full_day in hours_info:
                            # Check if the corresponding hours span indicates "Closed"
                            if 'Closed' in hours_span.text:
                                hours_info[full_day] = 'Closed'
                            else:
                                # Extract hours for non-closed days
                                hours_info[full_day] = hours_span.text.strip()
                else:
                    print("Unexpected hours format. Ensure that each day has corresponding hours.")

            # Handle 24-hour information if it exists
            else:
                hours_element = next_steps_module.find('span', {'class': 'result-geo-hours'})
                if hours_element:
                    # Get 24-hour text strip
                    hours_text = hours_element.get_text(strip=True)

                    # Assuming the presence of this text means 24-hour operation
                    if '24' in hours_text or '24-hour' in hours_text.lower():
                        hours_info['24_hour'] = True
                    else:
                        print("Unable to determine if the office is 24 hours.")
                else:
                    print("Hours information not available.")

            # Output the result
            print(hours_info)
    
    
            # Loop through the found <li> elements and extract labels and values
            Extra_element = li_element.find('div', {'class': 'panel-wrapper more-info-panel'})
            elig = []
            eligibility_rules_element = Extra_element.find('div', {'class': 'eligibility-rules'})
    
            # Check if eligibility_rules_element is found
            if eligibility_rules_element:
                # Check if eligibility_rules_element contains a list (ul)
                ul_element = eligibility_rules_element.find('ul')
        
                if ul_element:
                    # If it contains a list, extract list items and store them in a flat list
                    eligibility_list = [li.text.strip() for li in ul_element.find_all('li')]
                    elig.extend(eligibility_list)  # Use extend to add elements to the list directly
                    #print("Eligibility:", eligibility_list)
                else:
                    # If it doesn't contain a list, store the text as is
                    eligibility_text = eligibility_rules_element.text.strip()
                    #print("Eligibility:", [eligibility_text])  # Wrap in a list to maintain consistency
                    elig.append(eligibility_text)

            else:
                # Skip if 'eligibility-rules' class is not found
                pass
    
        
            print("Eligibility:", elig)
    
            # Extract Availability
            availability_element = Extra_element.find('strong', {'data-translate': 'Availability'})
            availability = availability_element.find_next('div', {'class': 'col-md-10'}).text.strip()

            # Extract Description
            description_element = Extra_element.find('strong', {'data-translate': 'Description'})
            description = description_element.find_next('div', {'class': 'col-md-10'}).text.strip()

            # Extract Languages
            languages_element = Extra_element.find('strong', {'data-translate': 'Languages'})
            languages = languages_element.find_next('div', {'class': 'col-md-10'}).text.strip()
            languages_data = [lang.strip() for lang in languages.split(',')]
            
            # Extract Cost
            cost_element = Extra_element.find('strong', string='Cost:')
            cost = cost_element.find_next('div', {'class': 'col-md-10'}).text.strip()
            
            # Extract Website URLs if they exist, or set them to None
            website_element = Extra_element.find('div', {'data-translation': 'Website'})
            website_url = website_element.find_next('a', {'class': 'activity-log descriptionProgramWebsite'})['href'] if website_element else None

            # Extract Facebook and Twitter URLs if they exist, or set them to None
            facebook_element = Extra_element.find('strong', {'data-translate': 'Facebook'})
            facebook_url = facebook_element.find_next('a', {'class': 'activity-log descriptionProgramFacebook'})['href'] if facebook_element else None

            twitter_element = Extra_element.find('strong', {'data-translate': 'Twitter'})
            twitter_url = twitter_element.find_next('a', {'class': 'activity-log descriptionProgramTwitter'})['href'] if twitter_element else None

            # Extract Coverage Area
            coverage_element = Extra_element.find('strong', {'data-translate': 'Coverage Area'})
            coverage = coverage_element.find_next('div', {'class': 'col-md-10'}).text.strip()
            
            # Initialize latitude and longitude as None by default
            latitude = None
            longitude = None
            zipcode = None

            # Find the element with class "office-hour-address"
            location_element = Extra_element.find('div', {'class': 'office-hours-address _js_address address notranslate'})

            # Check if the element exists and has the required attributes
            if location_element:
                latitude = location_element['data-latitude'] if location_element.has_attr('data-latitude') else None
                longitude = location_element['data-longitude'] if location_element.has_attr('data-longitude') else None
                print("Latitude:", latitude)
                print("Longitude:", longitude)
                
                # Extract all text within location_element
                address_text = location_element.get_text(separator=" ").strip()  # Get all text as a single string

                # Use regular expression to search for the ZIP code pattern anywhere in the text
                zip_matches = re.findall(r'(?<!\d)(\b\d{5}\b)(?!\d)', address_text)
                zipcode = int(zip_matches[-1]) if zip_matches else None
                print("ZIP Code:", zipcode)

            # Print or use the extracted values as needed
            print("Availability:", availability)
            print("Description:", description)
            print("Languages:", languages_data)
            print("Cost:", cost)
            print("Facebook URL:", facebook_url)
            print("Twitter URL:", twitter_url)
            print("Coverage Area:", coverage)

            data.append([
            Service_name,
            Service_url,
            main_s,
            other_s,
            serving_,
            phone_number,
            website_url,
            location_address,
            location_url_map,
            elig,
            availability,
            description,
            languages_data,
            cost,
            is_reviewed,
            facebook_url,
            twitter_url,
            coverage,
            latitude,
            longitude,
            zipcode,
            hours_info['24_hour'],
            hours_info['Monday'],      
            hours_info['Tuesday'],     
            hours_info['Wednesday'],   
            hours_info['Thursday'],   
            hours_info['Friday'],     
            hours_info['Saturday'],  
            hours_info['Sunday']])    
            print("***************************************************************************")

Service name:  Food Distribution
URL:  https://www.auntbertha.com//boys-%2526-girls-club-of-greater-redlands-riverside--redlands-ca--food-distribution/4904578973827072?postal=92507
Reviewed on: 11/07/2024
Main Services: ['food pantry']
Other Services: None
Serving: ['anyone in need', 'all ages', 'individuals', 'families']
Phone Number: 888-822-6535
Location Address: 1251 Clay Street, Redlands, CA 92374
Location url map: https://www.google.com/maps/?q=1251+Clay+Street,+Redlands,+CA+92374/
{'24_hour': False, 'Monday': 'Closed', 'Tuesday': 'Closed', 'Wednesday': 'Closed', 'Thursday': '2:00 PM - 6:00 PM  PST', 'Friday': 'Closed', 'Saturday': 'Closed', 'Sunday': 'Closed'}
Eligibility: []
Latitude: 34.0697895
Longitude: -117.1849724
ZIP Code: 92374
Availability: available
Description: The Boys & Girls Club of Greater Redlands-Riverside provides a food distribution for individuals and families in the community who are in need.This program provides:- Food to meet basic nutritional needs
Langua

Service name:  Fish Food Bank
URL:  https://www.auntbertha.com//riverside-latino-commission--coachella-ca--fish-food-bank/4612467847200768?postal=92507
Reviewed on: 11/06/2024
Main Services: ['food pantry']
Other Services: None
Serving: ['anyone in need', 'all ages', 'individuals', 'families', 'low-income']
Phone Number: 760-398-9000
Location Address: None
Location url map: None
{'24_hour': False, 'Monday': 'Closed', 'Tuesday': 'Closed', 'Wednesday': '9:00 AM - 10:30 AM  PST', 'Thursday': 'Closed', 'Friday': '9:00 AM - 10:30 AM  PST', 'Saturday': 'Closed', 'Sunday': 'Closed'}
Eligibility: ['Anyone can access this program.']
Availability: available
Description: The Fish Food Bank provides quality food items to households experiencing food insecurity. This program supplies critical nutrition to hungry individuals and families.This program provides:- Food to meet basic nutritional needsPlease contact program before visiting for updated hours/information.
Languages: ['English', 'Spanish']


Service name:  Emergency Food Assistance Program
URL:  https://www.auntbertha.com//bright-path-center--riverside-ca--emergency-food-assistance-program/4510892700336128?postal=92507
Reviewed on: 11/06/2024
Main Services: ['food pantry', 'meals']
Other Services: None
Serving: ['anyone in need', 'all ages', 'unemployed', 'individuals', 'families', 'homeless', 'near homeless', 'low-income']
Phone Number: 951-682-0088
Location Address: 4505 Allstate Drive, Riverside, CA 92501
Location url map: https://www.google.com/maps/?q=4505+Allstate+Drive,+Riverside,+CA+92501/
{'24_hour': False, 'Monday': '9:00 AM - 5:00 AM  PST', 'Tuesday': '9:00 AM - 5:00 AM  PST', 'Wednesday': '9:00 AM - 5:00 AM  PST', 'Thursday': '9:00 AM - 5:00 AM  PST', 'Friday': '9:00 AM - 5:00 AM  PST', 'Saturday': 'Closed', 'Sunday': '2:00 PM - 5:00 PM  PST'}
Eligibility: ['This program serves anyone in need.']
Latitude: 34.0035875
Longitude: -117.3754911
ZIP Code: 92501
Availability: available
Description: Bright Path Center'

Service name:  Senior Feeding Program (CSFP)
URL:  https://www.auntbertha.com//find-food-bank--indio-ca--senior-feeding-program-%2528csfp%2529/6260108091326464?postal=92507
Reviewed on: 11/12/2024
Main Services: ['food pantry']
Other Services: ['emergency food', 'food pantry', 'nutrition education', 'financial assistance']
Serving: ['seniors', 'low-income', 'limited english']
Phone Number: 760-775-3663
Location Address: None
Location url map: None
{'24_hour': False, 'Monday': '8:00 AM - 5:00 PM', 'Tuesday': '8:00 AM - 5:00 PM', 'Wednesday': '8:00 AM - 5:00 PM', 'Thursday': '8:00 AM - 5:00 PM', 'Friday': '8:00 AM - 5:00 PM', 'Saturday': 'Closed', 'Sunday': 'Closed'}
Eligibility: ['Must be 60 years or older to qualify.', 'This program helps people with income at or below 130% of federal poverty guidelines.', 'Must reside within eastern Riverside and southern San Bernardino Counties.']
Availability: available
Description: CSFP works to improve the health of individuals at least 60 years o

Service name:  Food and Clothing Bank
URL:  https://www.auntbertha.com//upland-church-of-christ--upland-ca--food-and-clothing-bank/5306635295653888?postal=92507
Reviewed on: 11/08/2024
Main Services: ['food pantry', 'clothing']
Other Services: None
Serving: ['anyone in need', 'all ages', 'individuals', 'families']
No phone number found.
Location Address: 331 West 9th Street, Upland, CA 91786
Location url map: https://www.google.com/maps/?q=331+West+9th+Street,+Upland,+CA+91786/
{'24_hour': False, 'Monday': '10:00 AM - 11:00 AM  PST', 'Tuesday': 'Closed', 'Wednesday': 'Closed', 'Thursday': 'Closed', 'Friday': 'Closed', 'Saturday': 'Closed', 'Sunday': 'Closed'}
Eligibility: ['Live in Upland and the surrounding communities.', 'Photo ID']
Latitude: 34.0962711
Longitude: -117.6553154
ZIP Code: 91786
Availability: available
Description: We at the Upland Church of Christ care about the physical needs of our neighbors and our food pantry serves families and individuals in need twice a monthSer

Service name:  SoCal Reentry Program
URL:  https://www.auntbertha.com//tayba-foundation--san-bernardino-ca--socal-reentry-program/5164741974687744?postal=92507
Reviewed on: 11/08/2024
Main Services: ['food pantry', 'addiction & recovery', 'mental health care', 'group therapy', 'navigating the system', 'case management', 'one-on-one support', 'computer class', 'identification recovery']
Other Services: None
Serving: ['adults 18+', 'criminal justice history', 'unemployed', 'individuals', 'families', 'near homeless', 'limited english', 'all mental health', 'substance dependency']
Phone Number: 909-543-6173
Location Address: 1887 Business Center Drive South, San Bernardino, CA 92408
Location url map: https://www.google.com/maps/?q=1887+Business+Center+Drive+South,+San+Bernardino,+CA+92408/
{'24_hour': False, 'Monday': '9:00 AM - 5:00 PM  PST', 'Tuesday': '9:00 AM - 5:00 PM  PST', 'Wednesday': '9:00 AM - 5:00 PM  PST', 'Thursday': '9:00 AM - 5:00 PM  PST', 'Friday': '9:00 AM - 5:00 PM  PST'

Service name:  Food and Clothing Distribution
URL:  https://www.auntbertha.com//jordan-outreach-ministries--indio-ca--food-and-clothing-distribution/6338874625228800?postal=92507
Reviewed on: 11/12/2024
Main Services: ['emergency food', 'food pantry', 'diapers & formula', 'clothing', 'personal care items']
Other Services: None
Serving: ['all ages', 'individuals', 'families', 'low-income']
Phone Number: 760-398-3352
Location Address: 43160 Madison Street, Indio, CA 92201
Location url map: https://www.google.com/maps/?q=43160+Madison+Street,+Indio,+CA+92201/
{'24_hour': False, 'Monday': '9:00 AM - 3:00 PM  PST', 'Tuesday': '9:00 AM - 3:00 PM  PST', 'Wednesday': '9:00 AM - 3:00 PM  PST', 'Thursday': '9:00 AM - 3:00 PM  PST', 'Friday': '9:00 AM - 3:00 PM  PST', 'Saturday': 'Closed', 'Sunday': 'Closed'}
Eligibility: ['This program serves low income individuals and families in need.']
Latitude: 33.7357395
Longitude: -116.2499793
ZIP Code: 92201
Availability: available
Description: Jordan Out

Service name:  Community Pantry
URL:  https://www.auntbertha.com//kidz-come-first--riverside-ca--community-pantry/5439183731687424?postal=92507
Reviewed on: 11/09/2024
Main Services: ['food pantry', 'clothing']
Other Services: None
Serving: ['anyone in need', 'all ages']
Phone Number: 951-790-9331
Location Address: None
Location url map: None
{'24_hour': False, 'Monday': '9:00 AM - 5:00 PM  PST', 'Tuesday': '9:00 AM - 5:00 PM  PST', 'Wednesday': '9:00 AM - 5:00 PM  PST', 'Thursday': '9:00 AM - 5:00 PM  PST', 'Friday': '9:00 AM - 5:00 PM  PST', 'Saturday': 'Closed', 'Sunday': 'Closed'}
Eligibility: ['Anyone can access this program.']
Availability: available
Description: Kidz Come First Community Pantry supplies food and clothing to anyone in need in the community.This program provides:- Food to meet basic nutritional needs- Clothing Please reach out by phone for more information.
Languages: ['English']
Cost: Free
Facebook URL: None
Twitter URL: None
Coverage Area: This program covers re

In [4]:
# Save to CSV file 
#"FindHelp_extracted_data_riv_shelter.csv" is the name for the shelter csv
csv_filename = 'FindHelp_extracted_data_riv_food_pantry.csv'

with open(csv_filename, 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    
    # The header 
    header = [
        "Service_name",
        "Service_url",
        "Main_Services",
        "Other_Services",
        "Serving",
        "Phone_Number",
        "Website",
        "Location_Address",
        "Location_URL_Map",
        "Eligibility",
        "Availability",
        "Description",
        "Languages",
        "Cost",
        "Google_Review",
        "Facebook_URL",
        "Twitter_URL",
        "Coverage",
        "Latitude",
        "Longitude",
        "Zipcode",
        "24hour",
        "Monday",
        "Tuesday",
        "Wednesday",
        "Thursday",
        "Friday",
        "Saturday",
        "Sunday"
    ]
    csv_writer.writerow(header)
    
    csv_writer.writerows(data)

In [5]:
#"FindHelp_extracted_data_riv_shelter.csv" is the name for the shelter csv
df = pd.read_csv("FindHelp_extracted_data_riv_food_pantry.csv")

riv_zipcodes = [92501, 92502, 92503, 92504, 92505, 92506, 92507, 92508, 92513, 92514, 92516, 92517, 92521, 92522]

filtered_df = df[df['Zipcode'].isin(riv_zipcodes)]

filtered_df

,Service_name,Service_url,Main_Services,Other_Services,Serving,Phone_Number,Website,Location_Address,Location_URL_Map,Eligibility,...,Longitude,Zipcode,24hour,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
2,Food Pantry,https://www.auntbertha.com//elevate-life-churc...,['food pantry'],NaN,"['anyone in need', 'all ages', 'individuals', ...",NaN,https://elevatechurch.tv/food-distribution,"4017 Trailcreek Road, Riverside, CA 92505",https://www.google.com/maps/?q=4017+Trailcreek...,[],...,-117.491768,92505.0,False,Closed,9:00 AM - 11:00 AM PST,Closed,Closed,9:00 AM - 11:00 AM PST,Closed,Closed
5,Food Bank,https://www.auntbertha.com//lake-hills-christi...,['food pantry'],NaN,"['anyone in need', 'all ages', 'individuals', ...",951-737-6680,https://www.lhccr.com/copy-of-worship-team,"12500 Indiana Avenue, Riverside, CA 92503",https://www.google.com/maps/?q=12500+Indiana+A...,['Anyone can access these services.'],...,-117.496104,92503.0,False,Closed,Closed,Closed,Closed,11:00 AM - 1:00 PM PST,Closed,Closed
13,Blessing Bag of Food,https://www.auntbertha.com//trinity-lutheran-c...,['food pantry'],NaN,"['all ages', 'individuals', 'homeless']",NaN,http://trinityriverside.org/outreach/,"5969 Brockton Avenue, Riverside, CA 92506",https://www.google.com/maps/?q=5969+Brockton+A...,['This program serves homeless individuals.'],...,-117.396995,92506.0,False,8:00 AM - 12:00 PM PST,8:00 AM - 12:00 PM PST,8:00 AM - 12:00 PM PST,8:00 AM - 12:00 PM PST,8:00 AM - 12:00 PM PST,Closed,Closed
15,Grocery and Clothing Assistance,https://www.auntbertha.com//arlington-temporar...,"['food pantry', 'clothing', 'navigating the sy...",NaN,"['anyone in need', 'all ages']",951-689-5620,http://www.atariverside.org/,"10459 Arlington Avenue, Riverside, CA 92505",https://www.google.com/maps/?q=10459+Arlington...,['Anyone can access this program.'],...,-117.482447,92505.0,False,10:00 AM - 1:00 PM PST,10:00 AM - 1:00 PM PST,10:00 AM - 3:00 PM PST,10:00 AM - 1:00 PM PST,Closed,Closed,Closed
17,Food Pantry,https://www.auntbertha.com//queen-of-angels--r...,['food pantry'],NaN,"['anyone in need', 'all ages', 'individuals', ...",909-226-8980,https://queenofangelsriverside.com/food-pantry,"4824 Jones Avenue, Riverside, CA 92505",https://www.google.com/maps/?q=4824+Jones+Aven...,['Anyone can access this program.'],...,-117.478212,92505.0,False,Closed,Closed,Closed,8:00 AM - 10:00 AM PST,Closed,Closed,Closed
20,Emergency Food Assistance Program,https://www.auntbertha.com//bright-path-center...,"['food pantry', 'meals']",NaN,"['anyone in need', 'all ages', 'unemployed', '...",951-682-0088,https://brightpathcenter.org/emergency-food-se...,"4505 Allstate Drive, Riverside, CA 92501",https://www.google.com/maps/?q=4505+Allstate+D...,['This program serves anyone in need.'],...,-117.375491,92501.0,False,9:00 AM - 5:00 AM PST,9:00 AM - 5:00 AM PST,9:00 AM - 5:00 AM PST,9:00 AM - 5:00 AM PST,9:00 AM - 5:00 AM PST,Closed,2:00 PM - 5:00 PM PST
21,Food Pantry,https://www.auntbertha.com//community-settleme...,['food pantry'],NaN,"['anyone in need', 'all ages', 'individuals', ...",NaN,https://csariverside.org/food-pantry-program/,"4366 Bermuda Avenue, Riverside, CA 92507",https://www.google.com/maps/?q=4366+Bermuda+Av...,['Anyone can access this program.'],...,-117.362520,92507.0,False,10:30 AM - 12:30 PM PST,Closed,10:30 AM - 12:30 PM PST,Closed,Closed,Closed,Closed
22,Thanksgiving Food Giveaway,https://www.auntbertha.com//community-settleme...,"['food delivery', 'food pantry']",NaN,"['anyone in need', 'all ages', 'individuals', ...",951-686-6266,https://csariverside.org/,"4366 Bermuda Avenue, Riverside, CA 92507",https://www.google.com/maps/?q=4366+Bermuda+Av...,['Anyone can access this program.'],...,-117.362612,92507.0,False,3:00 PM - 4:30 PM PST,3:00 PM - 4:30 PM PST,3:00 PM - 4:30 PM PST,3:00 PM - 4:30 PM PST,3:00 PM - 4:30 PM PST,Closed,Closed
24,Support Services - Homeless Outreach,https://www.auntbertha.com//bright-path-center...,"['food pantry', 'help find housing', 'clo